In [1]:
import pandas as pd

def timeDifference(t1, t2):
  t1=pd.to_datetime(t1)
  t2=pd.to_datetime(t2)
  timeDif=abs(pd.Timedelta(t2-t1).seconds /3600 )
  return (timeDif<=1)




In [8]:
data = pd.read_csv('order_brush_order.csv',parse_dates=['event_time'])

data = data.sort_values(by=['shopid','event_time','userid'])

final_dict={}

totalRows=data.shape[0]
for i in range(totalRows):
    userid1=data.userid.iloc[i]
    shopid1=data.shopid.iloc[i]
    time1=data.event_time.iloc[i]
    
    orders=1
    unique_ids={userid1:1}
    if (shopid1 not in final_dict):
        final_dict.update({shopid1:[]})
    
    for j in range(i+1,totalRows):
        userid2=data.userid.iloc[j]
        shopid2=data.shopid.iloc[j]
        time2=data.event_time.iloc[j]
        isOneHour=timeDifference(time1,time2)
        
        if (shopid1!=shopid2 or not isOneHour):
            break
        
        orders+=1
        if userid2 not in unique_ids:
            unique_ids.update({userid2:1})
        else:
            unique_ids[userid2]+=1
    
    
    uniqueVisits=len(unique_ids)
    concentration=orders/uniqueVisits
    
    if (concentration>=3):
        sus_ids=[]
        unique_id_list=list(unique_ids.items())
        unique_id_list.sort(key=lambda x:x[1], reverse=True)
        highest_visit=unique_id_list[0][1]
        
        for visitors in unique_id_list:
            if visitors[1]==highest_visit:
                final_dict[shopid1].append(visitors[0])
            else:
                break

        

        

In [9]:
final_list=[]

for k,v in final_dict.items():
  if len(v)==0:
    sus='0'
  else:
    sus=''
    v.sort()
    for i in v:
      id=str(i)+"&"
      sus+=id
    sus=sus[:len(sus)-1]
  helper_list=[k,sus]
  final_list.append(helper_list)


In [10]:
shopids = []
userids = []
for shop in final_list:
  shopids.append(shop[0])
  userids.append(shop[1])



In [11]:
output = pd.DataFrame({'shopid':shopids,'userid':userids})

In [6]:
output.head()

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0


In [12]:
output.to_csv('test.csv')